In [1]:
import torch
import torchvision
import numpy as np
from torch import nn,optim
import pandas as pd

pre-processing data

In [2]:
train_data=pd.read_csv(r'F:\study\ml\DataSet\House_Prices\train.csv')
test_data=pd.read_csv(r'F:\study\ml\DataSet\House_Prices\test.csv')

In [3]:
all_features=pd.concat((train_data.iloc[:,1:-1],test_data.iloc[:,1:]))

In [4]:
all_features.shape

(2919, 79)

In [5]:
numeric_features=all_features.dtypes[all_features.dtypes != 'object'].index
numeric_features

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')

In [6]:
all_features[numeric_features]=all_features[numeric_features].apply(
lambda x: (x-x.mean()) / (x.std()))

In [7]:
all_features=all_features.fillna(0)

In [8]:
all_features.shape

(2919, 79)

In [9]:
all_features=pd.get_dummies(all_features,dummy_na=True)
all_features.shape

(2919, 354)

In [10]:
num_train=train_data.shape[0]
train_features=torch.Tensor(all_features.iloc[:num_train,:].values)
test_features=torch.Tensor(all_features.iloc[num_train:,:].values)
train_labels=torch.Tensor(train_data.SalePrice.values).view(-1,1)

define model and func

In [11]:
loss=nn.MSELoss()

In [12]:
def get_net(feature_num):
    net=nn.Linear(feature_num,1)
    for p in net.parameters():
        nn.init.normal_(p,mean=0,std=0.01)
    return net
    

In [13]:
def log_rmse(net,features,labels):
    with torch.no_grad():
        clipped_pred=torch.max(net(features),torch.Tensor([1.0]))
        rmse=torch.sqrt(2*loss(clipped_pred.log(),labels.log()).mean())
    return rmse.item()

In [14]:
def train(net,train_features,train_labels,test_features,test_labels,
         num_epochs,learning_rate,weight_decay,batch_size):
    net=net.float()
    optimizer=optim.Adam(net.parameters(),lr=learning_rate,weight_decay=weight_decay)
    train_ls,test_ls=[],[]
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    train_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    for i in range(num_epochs):
        for x,y in train_iter:
            l=loss(net(x).float(),y.float())
            
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            
        train_ls.append(log_rmse(net,train_features,train_labels))
        if test_labels is not None:
            test_ls.append(log_rmse(net,test_features,test_labels))
    return train_ls,test_ls
        
            

In [15]:
def get_k_fold_data(k,i,x,y):

    x_train,y_train=None,None
    fold_size=x.shape[0]//k
    for j in range(k):
        idx=slice(j*fold_size,(j+1) *fold_size)
        x_part,y_part=x[idx,:],y[idx]
        if i==j:
            x_valid,y_valid=x_part,y_part
        elif x_train is None:
            x_train,y_train=x_part,y_part
        else:
            x_train=torch.cat((x_train,x_part),dim=0)
            y_train=torch.cat((y_train,y_part),dim=0)
    return x_train,y_train,x_valid,y_valid

In [16]:
def k_fold(k,x_train,y_train,num_epochs,learning_rate,weight_decay,batch_size):
    train_l_sum,valid_l_sum=0,0
    net=get_net(x_train.shape[1])
    for i in range(k):
        data=get_k_fold_data(k,i,x_train,y_train)
        train_ls,valid_ls=train(net,*data,num_epochs,learning_rate,
                               weight_decay, batch_size)
        train_l_sum+=train_ls[-1]
        valid_l_sum+=valid_ls[-1]
        print('fold %d ,train rmse %f,valid rmse %f ' %(
        i+1,train_ls[-1],valid_ls[-1]))
    return train_l_sum/k,valid_l_sum/k
        
        

In [17]:
k,num_epochs,lr,weight_decay,batch_size=5,100,5,0,64

In [18]:
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr,
                          weight_decay, batch_size)
print('%d-fold validation: avg train rmse %f, avg valid rmse %f' %
      (k, train_l, valid_l))

fold 1 ,train rmse 0.240077,valid rmse 0.221725 
fold 2 ,train rmse 0.187932,valid rmse 0.207320 
fold 3 ,train rmse 0.175479,valid rmse 0.196727 
fold 4 ,train rmse 0.181101,valid rmse 0.175211 
fold 5 ,train rmse 0.169347,valid rmse 0.206272 
5-fold validation: avg train rmse 0.190787, avg valid rmse 0.201451


In [36]:
def train_and_pred(train_features, test_features, train_labels, test_data,
                   num_epochs, lr, weight_decay, batch_size):
    net = get_net(train_features.shape[1])
    train_ls, _ = train(net, train_features, train_labels, test_features, None,
                        num_epochs, lr, weight_decay, batch_size)
    print('train rmse %f' % train_ls[-1])
    preds = net(test_features).detach().numpy()
    test_data['SalePrice'] = pd.Series(preds.reshape(1, -1)[0])
    submission = pd.concat([test_data['Id'], test_data['SalePrice']], axis=1)
    submission.to_csv(r'F:\study\ml\DataSet\HousePrice\Submission1.csv',
                      index=False)

In [37]:
train_and_pred(train_features, test_features, train_labels, test_data,
               num_epochs, lr, weight_decay, batch_size)

train rmse 0.230025


In [29]:
a=np.array([1,2,3,4,5,6]).reshape(2,3)
print(a)
print(a.reshape(1,-1))
print(a.reshape(1,-1)[0])
pd.Series(a.reshape(1,-1)[0])

[[1 2 3]
 [4 5 6]]
[[1 2 3 4 5 6]]
[1 2 3 4 5 6]


0    1
1    2
2    3
3    4
4    5
5    6
dtype: int32

In [19]:
x=torch.Tensor([[2.01],[3.01]])
y=torch.Tensor([[4.01],[6.02]])
w=torch.Tensor(np.random.normal(0,0.01,(1,1)))
w.requires_grad_(requires_grad=True)

tensor([[-0.0048]], requires_grad=True)

In [20]:
# torch.mm(x,w)

In [21]:
def linreg_net(x,w):
    return torch.mm(x,w)

In [22]:
def sgd(params,lr):
    with torch.no_grad():
        print('p.grad',params.grad)
        print('p.data',params.data)
        params -= lr*params.grad


In [23]:
def squared_loss(y_hat,y):
    return (y_hat-y)**2

In [24]:
for i in range(10):    
    y_hat=linreg_net(x,w)
    l=squared_loss(y_hat,y).sum()
    print(l)
    if w.grad is not None:
        w.grad.data.zero_()
        print('----------',w.grad.data)
    l.backward()
    
    print('----------',w.grad.data)
    if w.grad is not None:
        print('w.grad :',w.grad)
        print('w.data',w.data)
        sgd(w,0.05)
    print('new w',w)


tensor(52.5724, grad_fn=<SumBackward0>)
---------- tensor([[-52.4865]])
w.grad : tensor([[-52.4865]])
w.data tensor([[-0.0048]])
p.grad tensor([[-52.4865]])
p.data tensor([[-0.0048]])
new w tensor([[2.6195]], requires_grad=True)
tensor(5.0529, grad_fn=<SumBackward0>)
---------- tensor([[0.]])
---------- tensor([[16.2719]])
w.grad : tensor([[16.2719]])
w.data tensor([[2.6195]])
p.grad tensor([[16.2719]])
p.data tensor([[2.6195]])
new w tensor([[1.8059]], requires_grad=True)
tensor(0.4857, grad_fn=<SumBackward0>)
---------- tensor([[0.]])
---------- tensor([[-5.0446]])
w.grad : tensor([[-5.0446]])
w.data tensor([[1.8059]])
p.grad tensor([[-5.0446]])
p.data tensor([[1.8059]])
new w tensor([[2.0582]], requires_grad=True)
tensor(0.0467, grad_fn=<SumBackward0>)
---------- tensor([[0.]])
---------- tensor([[1.5639]])
w.grad : tensor([[1.5639]])
w.data tensor([[2.0582]])
p.grad tensor([[1.5639]])
p.data tensor([[2.0582]])
new w tensor([[1.9800]], requires_grad=True)
tensor(0.0046, grad_fn=<Sum